# Item Collaborative Filtering on Residuals
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* The weights here are the cosine correlation between the two items
* This alpha is trained on the residuals of all previous alphas

In [1]:
name = "ItemCFResid";

In [2]:
using NBInclude
@nbinclude("ItemCFBase.ipynb");

## Compute cosine correlations

In [3]:
function get_correlation_matrix_outdir(residual_alphas)
    # if the matrix is already stored on disk, return its filepath
    # otherwise, regenerate the matrix and store it to disk
    outdir = "$name/$(hash(residual_alphas))"
    if ispath("../../data/alphas/$outdir")
        return outdir
    end

    @debug "generating similarity matrix for $residual_alphas"
    training = get_residuals("training", residual_alphas)
    R = sparse(
        training.user,
        training.item,
        training.rating,
        maximum(training.user),
        maximum(training.item),
    )
    S = zeros(maximum(training.item), maximum(training.item))

    norms = map(norm, eachslice(R, dims = 2))
    norms[norms.==0] .= 1 # prevent division by 0
    @tprogress Threads.@threads for i = 1:size(S)[1]
        S[:, i] = vec(R[:, i]' * R) ./ norms ./ norms[i]
    end

    S = convert.(Float32, S)        
    write_params(Dict("S" => S), outdir = outdir)
    outdir
end;

## Setup hyperparameters

In [4]:
downcast_to_int(x) = isinteger(x) ? Int(x) : x
item_alphas = ["ItemCF.$K" for K in downcast_to_int.([2^4, 2^6, 2^8, 2^10])]
alphas = ["UserItemBiases"; item_alphas]
item_cf_params = [[
        cf_params(
            name = "ItemCFResid.$K",
            training_residuals = alphas,
            validation_residuals = alphas,
            neighborhood_type = "abs",
            S = get_correlation_matrix_outdir(alphas),
            K = K,
            λ = rand(3),
        ) for K in downcast_to_int.([2^10, 2^4, 2^6, 2^8])
    ];
];

## Train models

In [ ]:
for param in item_cf_params
    optimize_model(param)
end

Progress: 100%|███████████████████████████| Time: 0:11:27 ( 0.69  s/it)
[ Debug: 20220101 23:44:30 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.254687068296818,-0.004682033291061444,-0.014777159555176477,-0.004079955939055466) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(0.7188712354880293,0.5745975276031845,0.4628593106188109,0.3681593499690388): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDat

Iter     Function value   Gradient norm 
     0     1.254687e+00     1.477716e-02
 * Current step size: 1.0
 * time: 0.026314973831176758
 * g(x): [-0.004682033291061444, -0.014777159555176477, -0.004079955939055466]
 * x: [0.3972347982344473, 0.24134141645399687, 0.5477712521986617]


Progress: 100%|███████████████████████████| Time: 0:11:16 ( 0.68  s/it)
[ Debug: 20220101 23:57:04 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2544326670964745,-0.004615390637098091,-0.014450579126225141,-0.004096096881560934) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(0.7297937239122202,0.5855099423094746,0.4428305311055635,0.3742713371526252): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDa